# Example of CART
### See [CART WITH PYTHON](https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/)

#### Define the Gini score (same as reminder in ID3) 

In [57]:
def gini_score(partitions, labels):
    # count the amount of samples (or instances) in the split point
    n_instances = float(sum([len(partition) for partition in partitions]))
    # sum the weighted Gini index for each group
    gini = 0.0
    for partition in partitions:
        # Take the number of instances in each partition
        size = float(len(partition))
        # safety check to avoid dividing by cero. gini = 0 for this partition
        if size == 0:
            continue
        else:
            p_2 = 0.0 # probability of occurence squared
            # for each posible level (or label) of the target feature
            for label in labels:
                # calculate the frequency of appearance of the label in the partition
                p = [instance[-1] for instance in partition].count(label) / size
                p_2 += p*p
            # increase the score by the weighted gini index of each label
            gini += (1.0-p_2)*(size/n_instances)
    return gini

In [58]:
# test Gini values
print(gini_score([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
print(gini_score([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))

0.5
0.0


In [59]:
# create partitions from a dataset, separating instances by certain feature
def test_split(feature, level, dataset):
    left, right = list(), list()
    for instance in dataset:
        if instance[feature] < level:
            left.append(instance)
        else:
            right.append(instance)
    return left, right

In [60]:
# select the best split point for a dataset
def get_split(dataset):
    labels = list(set(instance[-1] for instance in dataset))
    b_feature, b_level, b_score, b_partitions = 999, 999, 999, None
    # Iterate over each feature in the dataset (avoid target)
    for feature in range(len(dataset[0])-1):
        # Iterate over each instance in the dataset
        for instance in dataset:
            partitions = test_split(feature, instance[feature], dataset)
            gini = gini_score(partitions, labels)
            if gini < b_score:
                b_feature, b_level, b_score, b_partitions = feature, instance[feature], gini, partitions
    # return best feature, split level and children partitions for the node
    return {'feature': b_feature, 'level':b_level,'partitions':b_partitions}    

In [61]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['feature']+1), split['level']))

Split: [X1 < 6.642]


#### For each leaf in the tree, choose the most frequent label on the instances of the partition

In [62]:
# Create a leaf node with the most common target level
def to_terminal(partition):
    outcomes = [instance[-1] for instance in partition]
    return max(set(outcomes),key=outcomes.count)

In [63]:
# Testing above function
print("The terminal label of the above dataset is", to_terminal(dataset))

The terminal label of the above dataset is 0


#### Using a recursive procedure, build the tree while spliting the dataset in partitions. Avoid overfitting relaying on pre-prunning (max. depth and min. num. instances)

In [64]:
# Create split of the dataset to make a new node or a leaf
def split(node, max_depth, min_size, depth):
    # Take partitions and delete the key in the node dict. structure
    left, right = node['partitions']
    del(node['partitions'])
    # Check base conditions
    # 1. No more instances
    if not left or not right:
        node['left'] = node['right'] = to_terminal( left + right )
        return
    # 2. max. depth reached
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # Process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)      
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1 )
        # Process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)      
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1 )

In [93]:
# Build decision tree function
def build_tree( train_dataset, max_depth, min_size):
    # Start by taking the root node
    root = get_split(train_dataset)
    # Call split to traverse the rest of the tree
    split(root, max_depth, min_size, 1)
    # return tree
    return root

#### Testing with the above dataset

In [66]:
# Print a decision tree
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % ((depth*' ', (node['feature']+1), node['level'])))
		print_tree(node['left'], depth+1)
		print_tree(node['right'], depth+1)
	else:
		print('%s[%s]' % ((depth*' ', node)))

In [67]:
print_tree(build_tree(dataset,3,1))

[X1 < 6.642]
 [X1 < 2.771]
  [0]
  [X1 < 2.771]
   [0]
   [0]
 [X1 < 7.498]
  [X1 < 7.445]
   [1]
   [1]
  [X1 < 7.498]
   [1]
   [1]


#### Create predicting function (also recursive)

In [86]:
# Make a prediction with a decision tree
def predict(node, query):
    if query[node['feature']] < node['level']:
        # Check whether the leaf was reached
        if isinstance(node['left'],dict):
            return predict(node['left'],query)
        else:
            return node['left']
    else:
    # Check whether the leaf was reached
        if isinstance(node['right'],dict):
            return predict(node['right'],query)
        else:
            return node['right']

In [69]:
#  predict with a stump
stump = {'feature': 0, 'right': 1, 'level': 6.642287351, 'left': 0}
for instance in dataset:
	prediction = predict(stump, instance)
	print('Expected=%d, Got=%d' % (instance[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


### Let use the Banknote dataset

In [78]:
# Import necessary modules
from random import seed
from random import randrange
from csv import reader

In [79]:
# Load a CSV file helper function
def load_csv(filename):
    file = open(filename,'r+')
    lines = reader(file)
    dataset = list(lines)
    return dataset

In [80]:
# Convert string column to float helper function
def str_column_to_float(dataset,column):
    for instance in dataset:
        instance[column] = float(instance[column].strip())

In [81]:
# Split the dataset into a number of folds for validation
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

In [88]:
# Calculate accuracy percentage (how many instances of the evaluation dataset were correctly predicted)
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return (correct / float(len(actual))) * 100.0

In [83]:
# Function for evaluating the learnt model
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for instance in fold:
			instance_copy = list(instance)
			test_set.append(instance_copy)
			instance_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [instance[-1] for instance in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [84]:
# Classification and Regression Tree Algorithm
def decision_tree(train_dataset, test_dataset, max_depth, min_size):
	tree = build_tree(train_dataset, max_depth, min_size)
	predictions = list()
	for instance  in test_dataset:
		prediction = predict(tree, instance)
		predictions.append(prediction)
	return predictions

#### Test CART on Bank Note dataset

In [89]:
seed(1)
# load and prepare data
filename = 'data_banknote_authentication.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for column in range(len(dataset[0])):
	str_column_to_float(dataset, column)
# evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [96.35036496350365, 97.08029197080292, 97.44525547445255, 98.17518248175182, 97.44525547445255]
Mean Accuracy: 97.299%


# Doing the same with [random forest instead](https://github.com/llSourcell/random_forests/blob/master/Random%20Forests%20.ipynb)

In [91]:
# select the best split point for a dataset and apply feature select
def get_split_rf(dataset, n_features):
    labels = list(set(instance[-1] for instance in dataset))
    b_feature, b_level, b_score, b_partitions = 999, 999, 999, None
    features = list()
    # Select random features
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    # Iterate over each feature previously selected
    for feature in features:
        # Iterate over each instance in the dataset
        for instance in dataset:
            partitions = test_split(feature, instance[feature], dataset)
            gini = gini_score(partitions, labels)
            if gini < b_score:
                b_feature, b_level, b_score, b_partitions = feature, instance[feature], gini, partitions
    # return best feature, split level and children partitions for the node
    return {'feature': b_feature, 'level':b_level,'partitions':b_partitions}    

In [94]:
# Create split of the dataset to make a new node or a leaf for random forest
def split_rf(node, max_depth, min_size, n_features, depth):
    # Take partitions and delete the key in the node dict. structure
    left, right = node['partitions']
    del(node['partitions'])
    # Check base conditions
    # 1. No more instances
    if not left or not right:
        node['left'] = node['right'] = to_terminal( left + right )
        return
    # 2. max. depth reached
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # Process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)      
    else:
        node['left'] = get_split_rf(left, n_features)
        split_rf(node['left'], max_depth, min_size, n_features, depth+1 )
        # Process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)      
    else:
        node['right'] = get_split_rf(right, n_features)
        split_rf(node['right'], max_depth, min_size, n_features, depth+1 )

In [95]:
# Build decision tree function with feature select for random forest
def build_tree_rf( train_dataset, max_depth, min_size, n_features):
    # Start by taking the root node
    root = get_split_rf(train_dataset, n_features)
    # Call split to traverse the rest of the tree
    split_rf(root, max_depth, min_size, n_features, 1)
    # return tree
    return root

In [96]:
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
	sample = list()
    # Take a pecentage of the dataset for making the sample
	n_sample = round(len(dataset) * ratio)
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return sample

In [97]:
# Make a combined prediction
def bagging_predict(trees, query):
	predictions = [predict(tree, query) for tree in trees]
	return max(set(predictions), key=predictions.count)

In [105]:
# Model esemble: Random forest
# First, train the model using the instances in train_dataset
# Then predict the label of each instance in test_dataset
def random_forest(train_dataset, test_dataset, max_depth, min_size, sample_size, n_trees, n_features):
	trees = list()
	for i in range(n_trees):
		sample = subsample(train_dataset, sample_size)
		tree = build_tree_rf(sample, max_depth, min_size, n_features)
		trees.append(tree)
	predictions = [bagging_predict(trees, query) for query in test_dataset]
	return predictions

In [101]:
#square root function
from math import sqrt

In [106]:
# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10]:
	scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
	print('Trees: %d' % n_trees)
	print('Scores: %s' % scores)
	print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Trees: 1
Scores: [96.71532846715328, 97.44525547445255, 99.63503649635037, 94.16058394160584, 94.8905109489051]
Mean Accuracy: 96.569%
Trees: 5
Scores: [100.0, 98.54014598540147, 98.17518248175182, 98.17518248175182, 98.17518248175182]
Mean Accuracy: 98.613%
Trees: 10
Scores: [99.27007299270073, 99.63503649635037, 98.90510948905109, 98.54014598540147, 98.54014598540147]
Mean Accuracy: 98.978%
